In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [2]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

2.2.3
3.4.2


In [3]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time



In [4]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
"""
    Parameters
    ----------
    problem_formulation_id : int {0, ..., 5}
                             problem formulations differ with respect to the objectives
                             0: Total cost, and casualties
                             1: Expected damages, costs, and casualties
                             2: expected damages, dike investment costs, rfr costs, evacuation cost, and casualties
                             3: costs and casualties disaggregated over dike rings, and room for the river and evacuation costs
                             4: Expected damages, dike investment cost and casualties disaggregated over dike rings and room for the river and evacuation costs
                             5: disaggregate over time and space

    Notes
    -----
    problem formulations 4 and 5 rely on ArrayOutcomes and thus cannot straightforwardly
    be used in optimizations

    """
problem_formulation = 3
dike_model, planning_steps = get_model_for_problem_formulation(problem_formulation)

In [5]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.2_Bmax'], pff=False)
RealParameter('A.2_pfail', 0, 1, resolution=None, default=None, variable_name=['A.2_pfail'], pff=False)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.3_Bmax'], pff=False)
RealParameter('A.3_pfail', 0, 1, resolution=None, default=None, variable_name=['A.3_pfai

In [6]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False)
IntegerParameter('0_RfR 1', 0, 1, resolution=None, default=None, variable_name=['0_RfR 1'], pff=False)
IntegerParameter('0_RfR 2', 0, 1, resolution=None, default=None, variable_name=['0_RfR 2'], pff=False)
IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False)
IntegerParameter('1_RfR 1', 0, 1, resolution=None, default=None, variable_name=['1_RfR 1'], pff=False)
IntegerParameter('1_RfR 2', 0, 1, resolution=None, default=None, variable_name=['1_RfR 2'], pff=False)
IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False)
IntegerParameter('2_RfR 1', 0, 1, resolution=None, default=None, variable_name=['2_RfR 1'], pff=False)
IntegerParameter('2_RfR 2', 0, 1, resolution=None, default=None, variable_name=['2_RfR 2'], pff=False)
IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable

In [7]:
# enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('A.1 Total Costs', variable_name=('A.1_Expected Annual Damage', 'A.1_Dike Investment Costs'), function=<function sum_over at 0x0000027014FEA8C0>)
ScalarOutcome('A.1_Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths',), function=<function sum_over at 0x0000027014FEA8C0>)
ScalarOutcome('A.2 Total Costs', variable_name=('A.2_Expected Annual Damage', 'A.2_Dike Investment Costs'), function=<function sum_over at 0x0000027014FEA8C0>)
ScalarOutcome('A.2_Expected Number of Deaths', variable_name=('A.2_Expected Number of Deaths',), function=<function sum_over at 0x0000027014FEA8C0>)
ScalarOutcome('A.3 Total Costs', variable_name=('A.3_Expected Annual Damage', 'A.3_Dike Investment Costs'), function=<function sum_over at 0x0000027014FEA8C0>)
ScalarOutcome('A.3_Expected Number of Deaths', variable_name=('A.3_Expected Number of Deaths',), function=<function sum_over at 0x0000027014FEA8C0>)
ScalarOutcome('A.4 Total Costs', variable_name=('A.4_Expected Annual Dama

In [8]:
# running the model through EMA workbench
with SequentialEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=50, policies=4)

[MainProcess/INFO] performing 50 scenarios * 4 policies * 1 model(s) = 200 experiments
100%|████████████████████████████████████████| 200/200 [00:42<00:00,  4.73it/s]
[MainProcess/INFO] experiments finished


In [9]:
# observing the simulation runs
experiments, outcomes = results
print(outcomes.keys())
experiments

dict_keys(['A.1 Total Costs', 'A.1_Expected Number of Deaths', 'A.2 Total Costs', 'A.2_Expected Number of Deaths', 'A.3 Total Costs', 'A.3_Expected Number of Deaths', 'A.4 Total Costs', 'A.4_Expected Number of Deaths', 'A.5 Total Costs', 'A.5_Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs'])


,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model
0,108,86.313980,1.5,0.040858,264.995960,10.0,0.938765,295.967604,1.0,0.202847,...,8,2,7,1,10,10,0,4,0,dikesnet
1,116,236.962984,10.0,0.944224,171.920246,10.0,0.794791,321.343344,1.0,0.063537,...,8,2,7,1,10,10,0,5,0,dikesnet
2,126,294.443169,10.0,0.639651,178.911423,1.5,0.204917,246.792305,1.5,0.567192,...,8,2,7,1,10,10,0,6,0,dikesnet
3,120,68.499393,1.5,0.936385,228.338566,1.5,0.980717,77.636361,1.0,0.689881,...,8,2,7,1,10,10,0,7,0,dikesnet
4,46,230.778583,10.0,0.382703,269.626956,10.0,0.062822,32.145618,10.0,0.253493,...,8,2,7,1,10,10,0,8,0,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,26,142.513050,1.0,0.760665,93.758851,1.0,0.156168,239.937658,10.0,0.588926,...,4,8,0,10,8,1,1,49,3,dikesnet
196,131,279.991950,1.0,0.211001,160.579160,1.5,0.046650,104.693934,1.5,0.099073,...,4,8,0,10,8,1,1,50,3,dikesnet
197,82,102.999126,1.5,0.362229,63.937766,1.0,0.492271,153.064736,1.0,0.238863,...,4,8,0,10,8,1,1,51,3,dikesnet
198,8,218.613095,10.0,0.342971,327.085042,1.0,0.812049,286.412074,10.0,0.651564,...,4,8,0,10,8,1,1,52,3,dikesnet


In [10]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes)

,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,1.774167e+08,0.0,2.119660e+08,0.000000,1.510032e+08,0.000000,4.565905e+07,0.0,1.728587e+08,0.003127,1.051100e+09,0.000000
1,1.774167e+08,0.0,2.119660e+08,0.000000,1.510032e+08,0.000000,4.565905e+07,0.0,1.739267e+08,0.003156,1.051100e+09,0.000000
2,1.774167e+08,0.0,2.119660e+08,0.000000,1.510032e+08,0.000000,4.565905e+07,0.0,1.838358e+08,0.019781,1.051100e+09,0.000000
3,1.774167e+08,0.0,2.119660e+08,0.000000,1.510032e+08,0.000000,4.565905e+07,0.0,1.693924e+08,0.000000,1.051100e+09,0.000000
4,1.774167e+08,0.0,2.119660e+08,0.000000,1.510032e+08,0.000000,4.565905e+07,0.0,2.634692e+08,0.101331,1.051100e+09,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
195,1.709905e+08,0.0,2.831614e+08,0.000000,8.933976e+07,0.000000,2.812138e+07,0.0,1.718341e+08,0.000000,1.390100e+09,0.000000
196,1.709905e+08,0.0,2.878489e+08,0.001237,1.061536e+08,0.008147,2.812138e+07,0.0,1.718341e+08,0.000000,1.390100e+09,363.313694
197,1.709905e+08,0.0,2.831614e+08,0.000000,8.933976e+07,0.000000,2.812138e+07,0.0,1.718341e+08,0.000000,1.390100e+09,0.000000
198,1.709905e+08,0.0,2.831614e+08,0.000000,8.933976e+07,0.000000,2.812138e+07,0.0,1.718341e+08,0.000000,1.390100e+09,0.000000


In [ ]:
# defining specific policies
# for example, policy 1 is about extra protection in upper boundary
# policy 2 is about extra protection in lower boundary
# policy 3 is extra protection in random locations
# policy 4 doe niks
'''
A1 = Doesburg Upstream
A2 = Cortenoever Upmidstream
A3 = Zutphen Midstream
A4 = Gorssel Downmidstream
A5 = Deventer Downstream
0_RFR = Project Olburgen
1_RFR = Project Havikerwaard
2_RFR = project Tichelbeekse
3_RFR = Project Welsummer
4_RFR = Obstakelsverwijderen
dan 0 of 1 of 2 is de timestep waarin het wordt geddaan
'''



def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}


policies = [
    Policy(
        "policy 1",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "A.1_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "policy 2",
        **dict(
            get_do_nothing_dict(),
            **{"4_RfR 0": 1, "4_RfR 1": 1, "4_RfR 2": 1, "A.5_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "policy 3",
        **dict(
            get_do_nothing_dict(),
            **{"1_RfR 0": 1, "2_RfR 1": 1, "3_RfR 2": 1, "A.3_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "policy 4",
        **dict(
            get_do_nothing_dict(),
            
        )
    ),
]

In [42]:
# pass the policies list to EMA workbench experiment runs
n_scenarios = 10
with SequentialEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, policies)

[MainProcess/INFO] performing 10 scenarios * 4 policies * 1 model(s) = 40 experiments
100%|██████████████████████████████████████████| 40/40 [00:08<00:00,  4.65it/s]
[MainProcess/INFO] experiments finished


In [43]:
experiments, outcomes = results

In [44]:
# only works because we have scalar outcomes
outcomes
pd.set_option('display.max_columns', None)
pd.DataFrame(outcomes)

,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,5.397251e+07,0.000000,3.845995e+07,0.036637,6.336090e+07,0.113238,6.013753e+06,0.003114,4.776826e+07,0.043162,253800000.0,0.0
1,5.397251e+07,0.000000,2.073308e+08,0.212016,0.000000e+00,0.000000,0.000000e+00,0.000000,2.688243e+07,0.026938,253800000.0,0.0
2,5.397251e+07,0.000000,1.008217e+07,0.010616,1.307078e+09,2.368811,0.000000e+00,0.000000,0.000000e+00,0.000000,253800000.0,0.0
3,5.397251e+07,0.000000,5.720843e+08,0.594113,0.000000e+00,0.000000,1.300431e+07,0.006443,0.000000e+00,0.000000,253800000.0,0.0
4,5.397251e+07,0.000000,1.403973e+08,0.123321,9.374879e+08,1.528638,0.000000e+00,0.000000,0.000000e+00,0.000000,253800000.0,0.0
5,5.397251e+07,0.000000,1.672558e+08,0.156020,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,253800000.0,0.0
6,5.397251e+07,0.000000,1.186184e+09,0.861656,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,253800000.0,0.0
7,5.397251e+07,0.000000,1.701341e+07,0.019718,1.776095e+09,3.512969,7.672370e+06,0.003438,0.000000e+00,0.000000,253800000.0,0.0
8,5.397251e+07,0.000000,2.087865e+07,0.019799,5.982907e+08,1.015821,0.000000e+00,0.000000,5.116281e+08,0.430254,253800000.0,0.0
9,5.397251e+07,0.000000,4.749496e+07,0.051084,2.666391e+07,0.054117,5.140137e+07,0.026849,0.000000e+00,0.000000,253800000.0,0.0
